In [1]:
import numpy as np
from scipy.optimize import curve_fit
import statsmodels.api as sm
from statsmodels.iolib.table import (SimpleTable, default_txt_fmt)

# My progs
from read_ts import get_ts

In [2]:
# Coordinate time series
souname = "2250+190"
coordts = get_ts(souname, "../data/ts")

In [3]:
mjy = np.array(coordts["mjy"]-2015)
mjy = sm.add_constant(mjy)

# Fit the slope
# R.A.
mod_wls_ra = sm.WLS(coordts["dra"], mjy, weights=1./(coordts["ra_err"] ** 2))
res_wls_ra = mod_wls_ra.fit()

# Declination
mod_wls_dec = sm.WLS(coordts["ddec"], mjy, weights=1./(coordts["dec_err"] ** 2))
res_wls_dec = mod_wls_dec.fit()

# Predictions
fit_dra = res_wls_ra.fittedvalues
fit_ddec = res_wls_dec.fittedvalues

In [4]:
print("R.A.")
print(res_wls_ra.summary())

R.A.
                            WLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.002
Model:                            WLS   Adj. R-squared:                 -0.004
Method:                 Least Squares   F-statistic:                    0.3531
Date:                Thu, 07 May 2020   Prob (F-statistic):              0.553
Time:                        18:31:13   Log-Likelihood:                -360.08
No. Observations:                 167   AIC:                             724.2
Df Residuals:                     165   BIC:                             730.4
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.1213      0.114     -1.069    

In [5]:
print("Decl.")
print(res_wls_dec.summary())

Decl.
                            WLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.002
Model:                            WLS   Adj. R-squared:                 -0.004
Method:                 Least Squares   F-statistic:                    0.2824
Date:                Thu, 07 May 2020   Prob (F-statistic):              0.596
Time:                        18:31:13   Log-Likelihood:                -426.80
No. Observations:                 167   AIC:                             857.6
Df Residuals:                     165   BIC:                             863.8
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.2784      0.173      1.605   

In [6]:
pm_ra, pmra_err = res_wls_ra.params[1], res_wls_ra.bse[1]
pm_dec, pmdec_err = res_wls_dec.params[1], res_wls_dec.bse[1]

In [7]:
# Total proper motion
pm2 = pm_ra**2 + pm_dec**2
pm = np.sqrt(pm2)
pm_err = np.sqrt(pm_ra**2 * pmra_err**2 + 
                 pm_dec**2 * pmdec_err**2) / pm

print("PM: {:.4f} +/- {:.4f}".format(pm, pm_err))

PM: 0.0147 +/- 0.0201


In [8]:
# P.A.
import astropy.units as u
from astropy.coordinates import SkyCoord

cen = SkyCoord(0*u.deg, 0*u.deg, frame="icrs")
oft = SkyCoord(pm_ra*u.deg, pm_dec*u.deg, frame="icrs")

pa = cen.position_angle(oft)
pa_deg = pa.to(u.deg).value

# Formal error
pa_err = np.sqrt(pm_dec**2 * pmra_err**2 + 
                 pm_ra**2 * pmdec_err**2) / pm2
pa_err_deg = np.rad2deg(pa_err)

print("PA: {:.0f} +/- {:.0f}".format(pa_deg, pa_err_deg))

PA: 324 +/- 69


In [9]:
from numpy import sin, cos, sqrt

In [10]:
### TO-DO
### Calculate from the Matrix translation method
# spa2 = sin(pa)**2
# cpa2 = cos(pa)**2
# pm2 = pm**2

# par_mat = np.array([[spa2, pm2 * cpa2], 
#                   [cpa2, pm2 * spa2]])
# wgt_mat = np.linalg.inv(par_mat)

# oft_sig = np.array([pmra_err**2, pmdec_err**2])
# pmt_sig2 = np.dot(wgt_mat, oft_sig)
# # pmt_sig = sqrt(np.dot(wgt_mat, oft_sig))

# pmt_sig2

# sqrt(pmt_sig2[0])

# par_mat, wgt_mat

In [11]:
def func(t, pm, pa, dra0, ddec0):
    """
    """
    
    dra = pm * sin(pa) * t + dra0
    ddec = pm * cos(pa) * t + ddec0
    
    return np.concatenate((dra, ddec))

In [12]:
mjy = np.array(coordts["mjy"]-2015)

doft = np.concatenate((coordts["dra"], coordts["ddec"]))

In [13]:
cov = coordts["ra_err"] * coordts["dec_err"] * coordts["ra_dec_corr"]
N = len(cov)

var_err = np.diag(np.concatenate((coordts["ra_err"]**2, coordts["dec_err"]**2)))

for i, covi in enumerate(cov):
    var_err[i, i+N] = covi
    var_err[i+N, i] = covi

In [14]:
popt, pcov = curve_fit(func, mjy, doft, sigma=var_err, absolute_sigma=False, 
                       bounds=([0, 0, -np.inf, -np.inf], 
                               [np.inf, 2*np.pi, np.inf, np.inf]))

In [15]:
# Test the results
t = np.arange(1, 30000) * 0.1
pm, pa = 3.5, np.pi / 4
dra0, ddec0 = 0.3, 0.7

from scipy.stats import norm
f0 = func(t, pm, pa, dra0, ddec0)
noise = norm.rvs(scale=0.4, size=len(f0))
f1 = f0 + noise

popt, pcov = curve_fit(func, t, f1)

popt

array([3.50000067, 0.78539929, 0.30130429, 0.69841121])

Try with the VSH fitting

In [16]:
def rotation(X, r1, r2, r3):
    """

    Parameters
    ----------
    x : 1-D array
        (ra, dec) in radian
    """

    ra, dec = X
    dra = -r1 * cos(ra) * sin(dec) - r2 * sin(ra) * sin(dec) + \
           r3 * cos(dec)
    ddec = + r1 * sin(ra) - r2 * cos(ra)

    return np.concatenate((dra, ddec))

In [40]:
def rotation1(X, **kwargs):
    """

    Parameters
    ----------
    x : 1-D array
        (ra, dec) in radian
    """

    ra, dec = X
    dra = -locals["r1"] * cos(ra) * sin(dec) - locals["r2"] * sin(ra) * sin(dec) + \
           locals["r3"] * cos(dec)
    ddec = + locals["r1"] * sin(ra) - locals["r2"] * cos(ra)

    return np.concatenate((dra, ddec))

In [18]:
from my_progs.catalog.read_icrf import read_icrf3
from my_progs.catalog.pos_diff import radio_cat_diff_calc

icrf3sx = read_icrf3(wv="sx")
icrf3k = read_icrf3(wv="k")

oft = radio_cat_diff_calc(icrf3k, icrf3sx, sou_name="iers_name")

In [19]:
from astropy import units as u

ra = oft["ra"].to(u.rad)
dec = oft["dec"].to(u.rad)

In [20]:
pos = (ra, dec)
doft = np.concatenate((oft["dra"], oft["ddec"]))

cov = oft["dra_ddec_cov"]
N = len(cov)

var_err = np.diag(np.concatenate((oft["dra_err"]**2, oft["ddec_err"]**2)))

for i, covi in enumerate(cov):
    var_err[i, i+N] = covi
    var_err[i+N, i] = covi

In [21]:
doft.size

1586

In [42]:
popt, pcov = curve_fit(rotation, pos, doft, sigma=var_err, absolute_sigma=False)

In [34]:
popt, pcov

(array([-0.03861199, -0.03117128,  0.00299405]),
 array([[ 4.80399602e-04,  5.61544931e-07, -2.53444850e-06],
        [ 5.61544931e-07,  4.75886605e-04,  1.59718526e-05],
        [-2.53444850e-06,  1.59718526e-05,  1.95857805e-04]]))

In [24]:
sig1 = np.sqrt(pcov.diagonal())
sig1

array([0.02191802, 0.02181483, 0.01399492])

In [25]:
from my_progs.catalog.vsh_deg1_cor import vsh_deg01_fitting

In [26]:
# Transform columns into np.array
dra = np.array(oft["dra"])
ddec = np.array(oft["ddec"])
dra_err = np.array(oft["dra_err"])
ddec_err = np.array(oft["ddec_err"])
ra_rad = np.array(oft["ra"].to(u.radian))
dec_rad = np.array(oft["dec"].to(u.radian))
dra_ddec_cov = np.array(oft["dra_ddec_cov"])

# Transformation parameters
pmt, sig, corr = vsh_deg01_fitting(
    dra, ddec, ra_rad, dec_rad, dra_err, ddec_err,
    cov=dra_ddec_cov, elim_flag="None", fit_type="rotation")

In [27]:
pmt, sig

(array([-0.038612  , -0.03117128,  0.00299405]),
 array([0.02194576, 0.02184244, 0.01401264]))

In [28]:
# eval?

In [29]:
# for i in range(5):

#     locals()["t_{:d}".format(i)] 

# # x = []